# CH4 텍스트와 바이트

## 4.1 문자문제
* 문자열 (string) : 문자의 `열`
* 문자 (char) -> 유니코드 문자
    * 파이썬3 str
    * 파이썬2 unicode (str은 유니코드가 아니다)
* 코드 포인트 (code point) : 문자의 단위 원소
    * 10진수 0에서 1,114,111 까지의 숫자
    * `U+` 접두사를 붙여 4자리에서 6자리 사이의 16진수로 표현
    * ex) A -> U+0041
* 인코딩 / 디코딩
    * 인코딩 : 코드 포인트를 바이트 시퀀스로 변환하는 알고리즘
    * ex) A (U+0041) -> \x41 (UTF-8), \x41\x00 (UTF-16LE)
    * 디코딩 : 바이트 시퀀스를 코드 포인트로 변환

In [54]:
def print_format(str_expr):
    print(str_expr, eval(str_expr), sep=' : ')

In [2]:
s = 'café'
print_format('len(s)')

b = s.encode('utf8')
print_format('b')
print_format('len(b)')

print_format('b[0]') # c = 99
print_format('b[1]') # a = 97
print_format('b[2]') # f = 102
print_format('b[3]') # \xc3 = 12 * 16 + 3 = 195
print_format('b[4]') # \xa9 = 10 * 16 + 9 = 169

print_format("b.decode('utf8')")

len(s) : 4
b : b'caf\xc3\xa9'
len(b) : 5
b[0] : 99
b[1] : 97
b[2] : 102
b[3] : 195
b[4] : 169
b.decode('utf8') : café


In [3]:
print(b'\x63')
print(b'\x61')
print(b'\x66')
print(b'\x63' == b'c')

b'c'
b'a'
b'f'
True


## 4.2 바이트에 대한 기본 지식
* 이진 시퀀스형 : bytes, bytearray
    * bytes : 파이썬 3에서 사용하는 불변형 타입 (리터럴 표현식 ex. b'caf\xc3\xa9')
    * bytearray : 파이썬 2.6에 추가된 가변형 타입 (리터럴 표현식 없음)
    * 각 구성요소는 0~255 사이의 정수 (1byte = 8bits = 2^8)
    * 이진 시퀀스를 슬라이싱하면 동일한 이진 시퀀스 형이 생성 (str 빼고는 슬라이싱했을때 타입이 동일함)

In [4]:
cafe = bytes('café', encoding='utf_8')
print_format('cafe')
print_format('cafe[0]')
print_format('cafe[:1]')

cafe_arr = bytearray(cafe)
print_format('cafe_arr')
print_format('cafe_arr[-1:]')

cafe : b'caf\xc3\xa9'
cafe[0] : 99
cafe[:1] : b'c'
cafe_arr : bytearray(b'caf\xc3\xa9')
cafe_arr[-1:] : bytearray(b'\xa9')


In [5]:
print(cafe[1])
print(cafe_arr[-1])

97
169


In [6]:
print(type(cafe))
print(type(b'\xac'))
print(type(cafe_arr))

<class 'bytes'>
<class 'bytes'>
<class 'bytearray'>


* 바이트 출력 방법
    * 화면에 출력가능한 아스키 문자 (공백, 물결표~ 등)은 그대로 출력
    * 탭(\t), 개행문자(\n), 캐리지 리턴(\r), 백슬렉시(\\)
    * 나머지는 \x 이후 16진수로 출력 (null byte = \xc00)
* byte와 bytearray 는 str 에서 제공하는 메서드 대부분 지원
    * format(), format_map() 은 지원하지 않음
    * fromhex() 라는 클래스 메서드 지원 (str 에는 없음)

In [7]:
bytes.fromhex('63 61 66 c3 a9')

b'caf\xc3\xa9'

* byte, bytearray 객체 생성 방법
    1. str, encoding 키워드 인수
    2. 0~255 사이의 정수 반복 가능형
    3. 하나의 정수 인수, 인수로 받은 정수 갯수만큼 널 바이트로 초기화된 이진 시퀀스 생성 (파이썬 3.6 부터 제거)
    4. bytes, bytearray, memoryview, array.array 등 버퍼 프로토콜을 구현하는 객체

In [8]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2]) # 'h' 는 short int (16비트 = 2바이트) 정수
octets = bytes(numbers)
octets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

In [9]:
octets.decode('utf8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfe in position 0: invalid start byte

In [10]:
array.array('h', octets[:4])

array('h', [-2, -1])

### 4.2.1 구조체와 메모리 뷰
* struct 모듈
    * 패킹된 바이트를 다양한 형의 필드로 구성된 튜플로 분석
    * 튜플을 패킹된 바이트로 변환하는 함수 제공
    * bytes, bytearray, memoryview 객체와 함께 사용

In [11]:
import struct
fmt = '<3s3sHH' # < 는 리틀엔디언, 3s3s 는 3바이트 시퀀스 두 개, HH 는 16비트 정수 두개
with open('filter.gif', 'rb') as fp:
    img = memoryview(fp.read())
    
header = img[:10] # memoryview 는 슬라이싱 해도 memoryview
print(bytes(header))
print(struct.unpack(fmt, header)) # '<3s3sHH' 에 맞게 튜플로 분리
del header
del img

b'GIF89a\xee\x04\xcc\x03'
(b'GIF', b'89a', 1262, 972)


In [12]:
# \xcc\x03 을 16진수로 변환하면 972? => 바이트 단위로 거꾸로 연산하는듯
12 * pow(16, 0) + 12 * pow(16, 1) + 3 * pow(16, 2) + 0 * pow(16, 3)

972

## 4.3 기본 인코더 / 디코더
* 파이썬에는 100 여개의 코덱(인코더/디코더) 포함
    * open(), str.encode(), bytes.decode() 호출시, encoding 인수에 전달하여 사용

In [13]:
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'EL Niño'.encode(codec), sep='\t')

latin_1	b'EL Ni\xf1o'
utf_8	b'EL Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00L\x00 \x00N\x00i\x00\xf1\x00o\x00'


* gb2312, utf-8, utf-16 은 가변 길이, 다중바이트 인코딩

![그림4-1](pic4-1.png)

* utf-8 : 웹에서 8비트 인코딩을 하기 위해 가장 널리 사용하는 방식, 아스키코드와 하위 호환됨 (순수 아스키 텍스트는 UTF-8 코드)

## 4.4 인코딩 / 디코딩 문제 이해하기

### 4.4.1 UnicodeEncodeError 처리하기
* 텍스트를 바이트로 변환할때, 문자가 대상 인코딩에 정의되어 있지 않으면 발생

In [14]:
city = 'São Paulo'
print_format("city.encode('utf_8')")
print_format("city.encode('utf_16')")
print_format("city.encode('iso8859_1')")
print_format("city.encode('cp437')")

city.encode('utf_8') : b'S\xc3\xa3o Paulo'
city.encode('utf_16') : b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'
city.encode('iso8859_1') : b'S\xe3o Paulo'


UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [15]:
print_format("city.encode('cp437', errors='ignore')") # 에러나는 문자 건너뜀
print_format("city.encode('cp437', errors='replace')") # 에러나는 문자 ? 로 치환
print_format("city.encode('cp437', errors='xmlcharrefreplace')") # 에러나는 문자 xml 개체로 치환

city.encode('cp437', errors='ignore') : b'So Paulo'
city.encode('cp437', errors='replace') : b'S?o Paulo'
city.encode('cp437', errors='xmlcharrefreplace') : b'S&#227;o Paulo'


### 4.4.2 UnicodeDecodeError 처리하기
* 이진 시퀀스를 텍스트로 변환할 때, 정당한 문자로 변환할 수 없으면 발생
    * 일부 레거시 8비트 코덱('cp1252', 'iso8859_1', 'koi8_r' 등) 에서는 무작위 비트 배열도 쓰레기문자로 조용히 디코딩

In [16]:
octets = b'Montr\xe9al' # latin1 인코딩
print_format("octets.decode('cp1252')") # cp1252는 latin1의 상위 집합 => 정상 디코딩
print_format("octets.decode('iso8859_7')") # 그리스 코덱 => 엉뚱한 문자로 디코딩
print_format("octets.decode('koi8_r')") # 러시아어 코덱 => 엉뚱한 문자로 디코딩
print_format("octets.decode('utf_8')") # 디코딩 에러

octets.decode('cp1252') : Montréal
octets.decode('iso8859_7') : Montrιal
octets.decode('koi8_r') : MontrИal


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [17]:
print_format("octets.decode('utf_8', errors='replace')") # U+FFFD 로 치환

octets.decode('utf_8', errors='replace') : Montr�al


### 4.4.3 예상과 달리 인코딩된 모듈을 로딩할 때 발생하는 SyntaxError
* 파이썬 3부터는 UTF-8 가 기본 인코딩 방식, 비 UTF-8 인코딩을 인코딩 선언없이 사용할때 발생

In [20]:
!python ola.py
## 에러를 강제로 내는건 안됨....

Olá, Mundo!


* 코드 상에 비아스키 식별자 사용해아 하나?
    * 되도록 안하는게 좋다

In [28]:
정지용 = '정지용'
print(정지용)

정지용


### 4.4.4 바이트 시퀀스의 인코딩 방식을 알아내는 방법
* 인코딩 정보 없이 100% 완벽하게 찾아내는건 불가능
    * 경험과 통계를 이용하여 추정
    * Chardetect 패키지

### 4.4.5 BOM : 유용한 깨진 문자
뭔소린지 잘 모르겠음 ㅜㅜ
* UTF-16 으로 인코딩된 텍스트 앞에는 여분의 바이트 존재
    * 바이트 순서 표시(BOM), 인코딩한 인텔 CPU 리틀엔디언(little endian) 바이트 순서를 나타냄
    * 리틀엔디언 컴퓨터에서는 코드 포인트 최하위 바이트 가 먼저 나옴.
        * ZERO WIDTH NO-BREAK SPACE(U+FEFF) => b'\xff\xfe'
    * 빅엔디언 컴퓨터에서는 인코딩 순서가 반대 ('E' -> [0, 69])

In [22]:
u16 = 'El Niño'.encode('utf_16')
print(u16)

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


In [23]:
list(u16)

[255, 254, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

In [25]:
u16le = 'El Niño'.encode('utf_16le')
list(u16le) # BOM 없음

[69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

In [26]:
u16be = 'El Niño'.encode('utf_16be')
list(u16be) # BOM 없음

[0, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111]

* BOM 이 없다면 UTF-16BE로 인코딩으로 간주하는게 표준
* But, 인텔 x86 아키텍처가 리틀엔디언이므로 BOM 없어도 UTF-16LE 일수도 있음.
* UTF-8 은 이런문제 없음
    * 윈도우 애플리케이션의 경우, UTF-8 인코딩하면서 BOM 붙이기도 함 (U+FEFF => b'\xef\xbb\xbf')

## 4.5 텍스트 파일 다루기
* 유니코드 샌드위치 : 가능한 빨리 bytes를 str 로 변환
    * 서비스 로직에서는 str 객체로만 처리하는게 좋음

In [29]:
open('cafe.txt', 'w', encoding='utf_8').write('café')
open('cafe.txt').read()

'café'

In [ ]:
## 리눅스라서 에러 발생안함.. 기본 인코딩이 UTF-8

In [30]:
fp = open('cafe.txt', 'w', encoding='utf_8')
fp

<_io.TextIOWrapper name='cafe.txt' mode='w' encoding='utf_8'>

In [31]:
fp.write('café')

4

In [32]:
fp.close()

In [33]:
import os
os.stat('cafe.txt').st_size

5

In [34]:
fp2 = open('cafe.txt')
fp2

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='UTF-8'>

In [35]:
fp2.encoding

'UTF-8'

In [36]:
fp3 = open('cafe.txt', encoding='cp1252')
fp3.read()

'cafÃ©'

In [38]:
fp4 = open('cafe.txt', 'rb')
print(fp4) # BufferedReader
fp4.read()

<_io.BufferedReader name='cafe.txt'>


b'caf\xc3\xa9'

### 4.5.1 기본 인코딩 설정 : 정신 나간거 아냐?

In [40]:
import sys, locale

expressions = """
    locale.getpreferredencoding()
    type(my_file)
    my_file.encoding
    sys.stdout.isatty()
    sys.stdout.encoding
    sys.stdin.isatty()
    sys.stdin.encoding
    sys.stderr.isatty()
    sys.stderr.encoding
    sys.getdefaultencoding()
    sys.getfilesystemencoding()
"""

my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'UTF-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


In [42]:
!python default_encodings.py

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> True
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> True
            sys.stdin.encoding -> 'UTF-8'
           sys.stderr.isatty() -> True
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


In [41]:
!python default_encodings.py > encodings.log

In [ ]:
 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> True
            sys.stdin.encoding -> 'UTF-8'
           sys.stderr.isatty() -> True
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'

* locale.getpreferredencoding() 설정이 가장 중요!
* But, 기본 인코딩에 의존하지 않는 것이 가장 좋음.

## 4.6 제대로 비교하기 위해 유니코드 정규화하기
* 유니코드에는 결합문자가 있어 문자열 비교가 쉽지 않음.

In [43]:
s1 = 'café'
s2 = 'cafe\u0301' # U+0301 : Combining Acute Accent
print(s1, s2)
print(len(s1), len(s2))
s1 == s2

café café
4 5


False

유니코드 정규화를 통해 문제 해결!
* unicodedata.normalize() 함수
    * NFC : 코드 포인트를 조합하여 가장 짧은 동일 문자열 생성 (W3C 추천 형태)
    * NFD : 조합된 문자를 기본 문자와 결합 문자로 분리
    * NFKC, NFKD : 호환성 문자로 치환 처리 (둘의 차이는?)
        * NFKC - 호환 분해 후, 결합 / NFKD - 호환 분해

In [46]:
from unicodedata import normalize
s1 = 'café'
s2 = 'cafe\u0301'
print_format('len(s1), len(s2)')
print_format("len(normalize('NFC', s1)), len(normalize('NFC', s2))")
print_format("len(normalize('NFD', s1)), len(normalize('NFD', s2))")
print_format("normalize('NFC', s1) == normalize('NFC', s2)")
print_format("normalize('NFD', s1) == normalize('NFD', s2)")

len(s1), len(s2) : (4, 5)
len(normalize('NFC', s1)), len(normalize('NFC', s2)) : (4, 4)
len(normalize('NFD', s1)), len(normalize('NFD', s2)) : (5, 5)
normalize('NFC', s1) == normalize('NFC', s2) : True
normalize('NFD', s1) == normalize('NFD', s2) : True


In [47]:
print_format("s1 == normalize('NFC', s2)")
print_format("normalize('NFD', s1) == s2")

s1 == normalize('NFC', s2) : True
normalize('NFD', s1) == s2 : True


In [49]:
# 다른 문자로 정규화 되는 케이스
from unicodedata import normalize, name
ohm = '\u2126'
print_format('name(ohm)')
ohm_c = normalize('NFC', ohm)
print_format('name(ohm_c)')
print_format('ohm == ohm_c')
print_format("normalize('NFC', ohm) == normalize('NFC', ohm_c)")

name(ohm) : OHM SIGN
name(ohm_c) : GREEK CAPITAL LETTER OMEGA
ohm == ohm_c : False
normalize('NFC', ohm) == normalize('NFC', ohm_c) : True


In [51]:
print(ohm, ohm_c)
print(ohm.encode('utf-8'), ohm_c.encode('utf-8'))

Ω Ω
b'\xe2\x84\xa6' b'\xce\xa9'


In [58]:
# NFKC 사용 예제
from unicodedata import normalize, name
half = '½'
normalize('NFKD', half)

'1⁄2'

In [59]:
four_squared = '42'
normalize('NFKD', four_squared)

'42'

In [65]:
micro = 'µ'
micro_kc = normalize('NFKC', micro)
print(micro, micro_kc)
print(ord(micro), ord(micro_kc))
print((name(micro), name(micro_kc)))

µ μ
181 956
('MICRO SIGN', 'GREEK SMALL LETTER MU')


In [67]:
print(name(normalize('NFC', micro)))
print(name(normalize('NFD', micro)))
print(name(normalize('NFKC', micro)))
print(name(normalize('NFKD', micro)))
print(name(normalize('NFC', micro_kc)))
print(name(normalize('NFD', micro_kc)))

MICRO SIGN
MICRO SIGN
GREEK SMALL LETTER MU
GREEK SMALL LETTER MU
GREEK SMALL LETTER MU
GREEK SMALL LETTER MU


### 4.6.1 케이스 폴딩
* 모든 텍스트릴 소문자로 변환하는 연산
* str.casefold() 메서드를 이용
* str.lower() 랑 다른 문자를 반환하는 코드 포인트가 약 0.11%

In [72]:
micro = 'µ'
print(name(micro))
micro_cf = micro.casefold()
print(name(micro_cf))
print(micro, micro_cf)
print()

eszett = 'ẞ'
print(name(eszett))
eszett_cf = eszett.casefold()
print(eszett, eszett_cf)

MICRO SIGN
GREEK SMALL LETTER MU
µ μ

LATIN CAPITAL LETTER SHARP S
ẞ ss


### 4.6.2 정규화된 텍스트 매칭을 위한 유틸리티 함수
* NFC 혹은 NFC + casefold 적용하여 비교하는게 좋다

In [14]:
from unicodedata import normalize

def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)

def fold_equal(str1, str2):
    return normalize('NFC', str1).casefold() == normalize('NFC', str2).casefold()

In [74]:
s1 = 'café'
s2 = 'cafe\u0301'
s1 == s2

False

In [75]:
nfc_equal(s1, s2)

True

In [76]:
nfc_equal('A', 'a')

False

In [78]:
s3 = 'Straẞe'
s4 = 'strasse'
print_format('s3 == s4')
print_format("nfc_equal(s3, s4)")
print_format("fold_equal(s3, s4)")
print_format("fold_equal(s1, s2)")
print_format("fold_equal('A', 'a')")

s3 == s4 : False
nfc_equal(s3, s4) : False
fold_equal(s3, s4) : True
fold_equal(s1, s2) : True
fold_equal('A', 'a') : True


### 4.6.3 극단적인 '정규화' : 발음 구별 기호 제거하기

In [81]:
import unicodedata
import string

def shave_marks(txt):
    norm_txt = unicodedata.normalize('NFD', txt)
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c))
    return unicodedata.normalize('NFC', shaved)

In [82]:
s1 = 'café'
shave_marks(s1)

'cafe'

In [83]:
def shave_marks_latin(txt):
    norm_txt = unicodedata.normalize('NFD', txt)
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:
            continue
        keepers.append(c)
        if not unicodedata.combining(c):
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)

In [84]:
# 구현 포기..

## 4.7 유니코드 텍스트 정렬하기

In [19]:
fruits = ['caju', 'atemoia', 'cajá', 'aćai', 'acerola']
sorted(fruits)

['acerola', 'atemoia', 'aćai', 'caju', 'cajá']

* locale.strxfrm() 는 시스템 설정을 바꿔야 하는등 단점이 많음

### 4.7.1 유니코드 대조 알고리즘을 이용한 정렬
* PyUCA 라이브러리 이용 (미리 정의된 대조 테이블을 통해 비교)

$ pip install pyuca

In [22]:
import pyuca
coll = pyuca.Collator()
fruits = ['caju', 'atemoia', 'cajá', 'aćai', 'acerola']
sorted_fruits = sorted(fruits, key=coll.sort_key)
sorted_fruits

['aćai', 'acerola', 'atemoia', 'cajá', 'caju']

## 4.8 유니코드 데이터베이스
* 유니코드 표준은 수많은 구조화된 텍스트 파일의 형태로 완전한 데이터베이스 제공
    * 코드 포인트를 문자명으로 매핑하는 테이블
    * 문자에 대한 메타데이터, 각 문자의 연관 방법 (unicodedata 에 존재)

In [24]:
import unicodedata
import re

re_digit = re.compile(r'\d')
sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print('U+%04x' % ord(char),
          char.center(6),
          're_dig' if re_digit.match(char) else '-',
          'isdig' if char.isdigit() else '-',
          'isnum' if char.isnumeric() else '-',
          format(unicodedata.numeric(char), '5.2f'),
          unicodedata.name(char),
          sep='\t'
        )

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


In [ ]:
# re 모듈은 유니코드 잘 인식 못함 => regex 모듈이 유니코드를 더 잘 지원

## 4.9 이중 모드 str 및 bytes API
* str, byte 인수를 모두 받지만 다르게 동작하는 모듈이 있음
    * re 와 os

### 4.9.1 정규 표현식에서의 str과 bytes
* bytes 로 정규표현식을 만들면 아스키 문자만 매칭됨, str 로 정규표현식을 만들면 유니코드 숫자나 문자도 매칭

In [27]:
import re

re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef as 1729 = 1\u00b3 + 12\u00b3 = 9\u00b3 + 10\u00b3.")
text_bytes = text_str.encode('utf_8')

print('Text', repr(text_str), sep='\n    ')
print('Numbers')
print('    str  :', re_numbers_str.findall(text_str))
print('    bytes:', re_numbers_bytes.findall(text_bytes))
print('Words')
print('    str  :', re_words_str.findall(text_str))
print('    bytes:', re_words_bytes.findall(text_bytes))

Text
    'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
    str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
    bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
    str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
    bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


### 4.9.2 os 모듈 함수에서 str과 bytes
* 리눅스 커널은 유니코드를 모름, OS 파일명은 바이트 시퀀스로 구성되어 있으면 str 로 디코딩 불가
* str 인수를 호출하면 sys.getfilesystemencoding() 함수에 지정된 코덱으로 변환
* 이렇게 처리할수 없는 경우, bytes 인수를 os 함수에 전달하여 bytes 반환값을 사용

In [30]:
import os
os.listdir('.')

['digits-of-π.txt',
 'pic4-1.png',
 'encodings.log',
 'ola.py',
 '.ipynb_checkpoints',
 'CH4.Text and Byte.ipynb',
 'default_encodings.py',
 'cafe.txt',
 'filter.gif',
 'dummy']

In [31]:
os.listdir(b'.')

[b'digits-of-\xcf\x80.txt',
 b'pic4-1.png',
 b'encodings.log',
 b'ola.py',
 b'.ipynb_checkpoints',
 b'CH4.Text and Byte.ipynb',
 b'default_encodings.py',
 b'cafe.txt',
 b'filter.gif',
 b'dummy']

fsencode(파일명)
* 파일명이 str 형이면 sys.getfilesystemencoding() 이 반환한 코덱명을 이용하여 bytes 형으로 인코딩, byte 형이면 그대로 반환

fsdecode(파일명)
* 파일명이 bytes 형이면 sys.getfilesystemencoding() 이 반환한 코덱명을 이용하여 str 형으로 인코딩, str 형이면 그대로 반환

유닉스 계열 플랫폼에서는 예상치 않은 바이트 문제를 피하기 위해 surrogateescape 에러 처리기를 사용 (윈도우에서는 strict 에러처리기 사용)
* 디코딩할 수 없는 바이트를 U+DC00~U+DCFF 까지 코드 포인트로 치환

In [33]:
pi_name_bytes = os.listdir(b'.')[0]
pi_name_str = pi_name_bytes.decode('ascii', 'surrogateescape')
pi_name_str

'digits-of-\udccf\udc80.txt'

In [34]:
pi_name_str.encode('ascii', 'surrogateescape')

b'digits-of-\xcf\x80.txt'